<a href="https://colab.research.google.com/github/yusufsamsum/naiveBayesClassifier/blob/master/naiveBayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import csv
import math
import random
import operator
import time

import numpy as np
import pandas as pd
from numpy import loadtxt

In [0]:
test_features = pd.read_csv("question-4-test-features.csv", header = None)

In [0]:
train_features = pd.read_csv("question-4-train-features.csv", header = None)

In [0]:
test_labels = pd.read_csv("question-4-test-labels.csv", header = None)

In [0]:
train_labels = pd.read_csv("question-4-train-labels.csv", header = None)

In [0]:
train_features.head()

In [0]:
train_labels.head()
train_labels.to_numpy()

In [0]:
train_features.to_numpy()

In [0]:
medical_word_count = np.zeros([train_features.shape[1]])
space_word_count = np.zeros([train_features.shape[1]])
cryptology_word_count = np.zeros([train_features.shape[1]])
electronics_word_count = np.zeros([train_features.shape[1]])

In [0]:
def get_word_count(category, train_row):
    for i in range(train_row.size):
        category[i] += train_row[i]
    

In [0]:
number_of_medical = 0
number_of_space = 0
number_of_cryptology = 0
number_of_electronics = 0
for label_index in range(train_labels[0].size):
    
    if train_labels[0][label_index] == 0: # medical
        number_of_medical += 1
        get_word_count(medical_word_count, train_features.loc[label_index])
    elif train_labels[0][label_index] == 1: # space
        number_of_space += 1
        get_word_count(space_word_count, train_features.loc[label_index])
    elif train_labels[0][label_index] == 2: # cryptology
        number_of_cryptology += 1
        get_word_count(cryptology_word_count, train_features.loc[label_index])
    elif train_labels[0][label_index] == 3: # electronics
        number_of_electronics += 1
        get_word_count(electronics_word_count, train_features.loc[label_index])
    else:
        print("Something is wrong with the data")

In [0]:
## saving number of occurences for each word in this category
dataset = pd.DataFrame({'Word Count': medical_word_count})
dataset.to_csv(r'medical.csv', index = None, header = True)

In [0]:
## saving number of occurences for each word in this category
dataset = pd.DataFrame({'Word Count': space_word_count})
dataset.to_csv(r'space.csv', index = None, header = True)

In [0]:
## saving number of occurences for each word in this category
dataset = pd.DataFrame({'Word Count': cryptology_word_count})
dataset.to_csv(r'cryptology.csv', index = None, header = True)

In [0]:
## saving number of occurences for each word in this category
dataset = pd.DataFrame({'Word Count': electronics_word_count})
dataset.to_csv(r'electronics.csv', index = None, header = True)

In [0]:
## loading back the number of occurences for each word in this category
medical_df = pd.read_csv('medical.csv')

for index, value in medical_df.iterrows():
    medical_word_count[index] = value
    pass
medical_word_count.size

In [0]:
## loading back the number of occurences for each word in this category
space_df = pd.read_csv('space.csv')

for index, value in space_df.iterrows():
    space_word_count[index] = value
    pass
space_word_count.size

In [0]:
## loading back the number of occurences for each word in this category
cryptology_df = pd.read_csv('cryptology.csv')

for index, value in cryptology_df.iterrows():
    cryptology_word_count[index] = value
    pass
cryptology_word_count.size

In [0]:
## loading back the number of occurences for each word in this category
electronics_df = pd.read_csv('electronics.csv')

for index, value in electronics_df.iterrows():
    electronics_word_count[index] = value
    pass
electronics_word_count.size

In [0]:
def get_total_word_count(word_count):
    total_word_count = 0
    for i in range(word_count.size):
        total_word_count += word_count[i]
        pass
    return total_word_count

In [0]:
total_medical_word_count = get_total_word_count(medical_word_count)
total_space_word_count = get_total_word_count(space_word_count)
total_cryptology_word_count = get_total_word_count(cryptology_word_count)
total_electronics_word_count = get_total_word_count(electronics_word_count)

In [0]:
#These values actually found by code in 11th cell
number_of_medical = 800
number_of_space = 800
number_of_cryptology = 800
number_of_electronics = 800

In [0]:
### prediction with mle
words_mle = np.zeros([train_features.shape[1], 4])

def special_divide(nominator, denominator):
    if denominator == 0:
        return 0
    return nominator / denominator


In [0]:
## log P(Xj | Y = yk )
for i in range(train_features.shape[1]):
    words_mle[i][0] = (special_divide(medical_word_count[i], total_medical_word_count))  ## medical
    words_mle[i][1] = (special_divide(space_word_count[i], total_space_word_count))     ## space
    words_mle[i][2] = (special_divide(cryptology_word_count[i], total_cryptology_word_count))  ## cryptology
    words_mle[i][3] = (special_divide(electronics_word_count[i], total_electronics_word_count)) ## electronics
    
words_mle

In [0]:
## get log P( Y = yk )
categories_prob = np.zeros(4)
total_train_mails = train_features.shape[0]
categories_prob[0] = np.log((number_of_medical / total_train_mails))
categories_prob[1] = np.log((number_of_space / total_train_mails))
categories_prob[2] = np.log((number_of_cryptology / total_train_mails))
categories_prob[3] = np.log((number_of_electronics / total_train_mails))
categories_prob.size

In [0]:
def product(x, y):
    if y == 0:
        if x != 0:
            return -np.inf
        else:
            return 0
            pass
        pass
    else:
        return x * math.log(y)

In [0]:
def classifier(test_data, category_prob, words):
    inner_sums = []
    max_value = -np.inf
    category = 1
    for i  in range(category_prob.size):
        inner_sum = 0
        value = 0
        for index in range(test_data.shape[0]):
            value = product(test_data[index], words[index][i])
            if  value == -np.inf:
                break
            else:
                inner_sum += value
        if value == -np.inf:
            continue
        inner_sum += category_prob[i]
        inner_sums.append(inner_sum)
        if max_value < inner_sum:
            max_value = inner_sum
            category = i
            
    #print(category)
    return category, inner_sums

def calc_accuracy(actual_values, predictions):
    correct = 0
    for i in range(len(predictions)):
        if predictions[i][0] == actual_values[i]:
            correct += 1
        
            
    return (correct/float(actual_values.size)*100)

In [0]:
results = []
for i in range(test_features.shape[0]):
    results.append(classifier(test_features.loc[i], categories_prob, words_mle))

In [0]:
actual_values = np.zeros([test_labels.size])

In [0]:
for i in  range(actual_values.shape[0]):
    actual_values[i] = test_labels.loc[i]

In [0]:
acc = calc_accuracy(actual_values, results)
acc

In [0]:
### saving results on another array
category_mle = []
for i in range(len(results)):
    category_mle.append(results[i][0])

In [0]:
### saving mle predictions on csv file
dataset = pd.DataFrame({'Category': category_mle})
dataset.to_csv(r'prediction_mle.csv', index = None, header = True)

In [0]:
### prediction with map ### 
words_map = np.zeros([train_features.shape[1], 4])


def drichlet_prior(nominator, denominator, alfa, length):
    return special_divide(nominator + alfa, denominator + alfa * length)

In [0]:
## log P(Xj | Y = yk )
for i in range(train_features.shape[1]):
    words_map[i][0] = drichlet_prior(medical_word_count[i], total_medical_word_count, 1, train_features.shape[1])  ## medical
    words_map[i][1] = drichlet_prior(space_word_count[i], total_space_word_count, 1, train_features.shape[1])      ## space
    words_map[i][2] = drichlet_prior(cryptology_word_count[i], total_cryptology_word_count, 1, train_features.shape[1])   ## cryptology
    words_map[i][3] = drichlet_prior(electronics_word_count[i], total_electronics_word_count, 1, train_features.shape[1]) ## electronics
    

In [0]:
results = []
for i in range(test_features.shape[0]):
    results.append(classifier(test_features.loc[i], categories_prob, words_map))

In [0]:
acc = calc_accuracy(actual_values, results)
acc

In [0]:
### saving results on another array
category_map = []
for i in range(len(results)):
    category_map.append(results[i][0])

In [0]:
### saving MAP predictions on csv file
dataset = pd.DataFrame({'Category': category_map})
dataset.to_csv(r'prediction_map.csv', index = None, header = True)

In [0]:
#### Confusion Matrix for mle and map predictions  ####
## You can read the prediction values from these csv files.
predictions_mle = np.zeros(800)
predictions_map = np.zeros(800)

df = pd.read_csv("prediction_mle.csv")

for index, value in df.iterrows():
    predictions_mle[index] = value
    
df = pd.read_csv("prediction_map.csv")


for index, value in df.iterrows():
    predictions_map[index] = value

In [0]:
def get_confusion_matrix(predictions, actual_values):
    cm = np.zeros((4,4))
    for pred, act in zip(predictions, actual_values):
        if act == 0:
            if pred == 0:
                cm[0][0] += 1
            elif pred == 1:
                cm[0][1] += 1
            elif pred == 2:
                cm[0][2] += 1
            elif pred == 3:
                cm[0][3] += 1
        elif act == 1:
            if pred == 0:
                cm[1][0] += 1
            elif pred == 1:
                cm[1][1] += 1
            elif pred == 2:
                cm[1][2] += 1
            elif pred == 3:
                cm[1][3] += 1
        elif act == 2:
            if pred == 0:
                cm[2][0] += 1
            elif pred == 1:
                cm[2][1] += 1
            elif pred == 2:
                cm[2][2] += 1
            elif pred == 3:
                cm[2][3] += 1
        else:
            if pred == 0:
                cm[3][0] += 1
            elif pred == 1:
                cm[3][1] += 1
            elif pred == 2:
                cm[3][2] += 1
            elif pred == 3:
                cm[3][3] += 1
    return cm

In [0]:
### confusion matrix for predictions with mle approach
cm_mle = get_confusion_matrix(predictions_mle, actual_values)
cm_mle

In [0]:
### confusion matrix for predictions with map approach
cm_map = get_confusion_matrix(predictions_map, actual_values)
cm_map

In [0]:
def print_smallest_or_highest_probs(results, kind):
    for i in range(len(results)):
        results[i][1].sort(reverse = kind)
    probs = []
    for i in range(len(results)):
        probs.append((i, results[i][0], results[i][1][0]))
    probs.sort(key = operator.itemgetter(2), reverse = kind)
    med = False 
    space = False 
    cryp = False 
    elec = False
    for i in range(len(probs)):
        if probs[i][1] == 0 and med == False:
            print( "medicine - index: " , probs[i][0], " probabilities: ", results[probs[i][0]][1])
            med = True
        elif probs[i][1] == 1 and space == False:
            print( "space - index: " , probs[i][0], " probabilities: ", results[probs[i][0]][1])
            space = True
        elif probs[i][1] == 2 and cryp == False:
            print( "cryptology - index: " , probs[i][0], " probabilities: ", results[probs[i][0]][1] )
            cryp = True
        elif probs[i][1] == 3 and elec == False:
            print( "electronics - index: " , probs[i][0], " probabilities: ", results[probs[i][0]][1])
            elec = True

In [0]:
#it is for MAP estimation results, therefore be careful when you run and be sure you are running the results which are found by MAP values
#print_smallest_or_highest_probs(results, True)
#print_smallest_or_highest_probs(results, False)

In [0]:
file = open('question-4-vocab.txt', 'r')
vocab = file.read().splitlines()
file.close()

In [0]:
def get_words_sorted_with_index(words, kind):
    sorted_words = []
    for i in range(words.shape[0]):
        sorted_words.append((i, words[i]))
    sorted_words.sort(key = operator.itemgetter(1), reverse = kind)
    return sorted_words

def get_top_20(words, vocab):
    top_20 = []
    for i in range(20):
        top_20.append(vocab[words[i][0]])
    return top_20

In [0]:
 ## the parameter kind == true it will use the highest probable words if it is false it will find the lowest probable words
medical_words_sorted = get_words_sorted_with_index(medical_word_count, True)
space_words_sorted = get_words_sorted_with_index(space_word_count, True)
cryptology_words_sorted = get_words_sorted_with_index(cryptology_word_count, True)
electronics_words_sorted = get_words_sorted_with_index(electronics_word_count, True)

# Please uncomment the code for finding lowest probable words

#medical_words_sorted = get_words_sorted_with_index(medical_word_count, False)
#space_words_sorted = get_words_sorted_with_index(space_word_count, False)
#cryptology_words_sorted = get_words_sorted_with_index(cryptology_word_count, False)
#electronics_words_sorted = get_words_sorted_with_index(electronics_word_count, False)

In [0]:
medical_top = get_top_20(medical_words_sorted, vocab)
space_top = get_top_20(space_words_sorted, vocab)
cryptology_top = get_top_20(cryptology_words_sorted, vocab)
electronics_top = get_top_20(electronics_words_sorted, vocab)

In [0]:
tops = [medical_top, space_top, cryptology_top, electronics_top]
for top in tops:
    for i in top:
        print(i)
    print("\n")